### **Fahimul Islam**

### **2017331026**

### **Absent/Present Predictor**

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import pprint
# Get markov edges
def get_markov_edges(df):
    # Create a dictionary
    edges = {}
    # Loop columns
    for column in df.columns:
        # Loop rows
        for row in df.index:
            edges[(row,column)] = df.loc[row,column]
    # Return edges
    return edges

def viterbi(pi, a, b, obs):
    
    nStates = np.shape(b)[0]
    T = np.shape(obs)[0]
    
    path = np.zeros(T)
    delta = np.zeros((nStates, T))
    phi = np.zeros((nStates, T))
    
    delta[:, 0] = pi * b[:, obs[0]]
    phi[:, 0] = 0
    for t in range(1, T):
        for s in range(nStates):
            delta[s, t] = np.max(delta[:, t-1] * a[:, s]) * b[s, obs[t]] 
            phi[s, t] = np.argmax(delta[:, t-1] * a[:, s])
    
    path[T-1] = np.argmax(delta[:, T-1])
    for t in range(T-2, -1, -1):
        path[t] = phi[int(path[t+1]),int(t+1)]
        
    return path, delta, phi
# The main entry point for this module
def main():
    # Observation states
    # The director can have an umbrella or not have an umbrella (equally likely)
    observation_states = ['BoringClass', 'InterestingClass']
    # Create hidden states with probabilities (250 rainy days per year)
    p = [0.32, 0.68]
    #p = [0.5, 0.5]
    #p = [0.7, 0.3]
    hidden_states = ['Present', 'Absent']
    state_space = pd.Series(p, index=hidden_states, name='states')
    # Print hidden states
    print('--- Hidden states ---')
    print(state_space)
    print()
    # Create a hidden states transition matrix with probabilities
    hidden_df = pd.DataFrame(columns=hidden_states, index=hidden_states)
    hidden_df.loc[hidden_states[0]] = [0.75, 0.25]
    hidden_df.loc[hidden_states[1]] = [0.25, 0.75]
    # Print transition matrix
    print('--- Transition matrix for hidden states ---')
    print(hidden_df)
    print()
    print(hidden_df.sum(axis=1))
    print()
    # Create matrix of observations with sensor probabilities
    observations_df = pd.DataFrame(columns=observation_states, index=hidden_states)
    observations_df.loc[hidden_states[0]] = [0.1, 0.9]
    observations_df.loc[hidden_states[1]] = [0.8, 0.2]
    # Print observation matrix
    print('--- Sensor matrix ---')
    print(observations_df)
    print()
    print(observations_df.sum(axis=1))
    print()
    # Create graph edges and weights
    hidden_edges = get_markov_edges(hidden_df)
    observation_edges = get_markov_edges(observations_df)
    # Print edges
    print('--- Hidden edges ---')
    pprint.pprint(hidden_edges)
    print()
    print('--- Sensor edges ---')
    pprint.pprint(observation_edges)
    print()
    # Observations
    observations_map = {0:'BoringClass', 1:'InterestingClass'}
    observations = np.array([1,1,1,0,1,1,1,0,0,0])
    observerations_path = [observations_map[v] for v in list(observations)]
    # Get predictions with the viterbi algorithm
    path, delta, phi = viterbi(p, hidden_df.values, observations_df.values, observations)
    state_map = {0:'Present', 1:'Absent'}
    state_path = [state_map[v] for v in path]
    state_delta = np.amax(delta, axis=0)
    # Print predictions
    print('--- Predictions ---')
    print(pd.DataFrame().assign(Observation=observerations_path).assign(Prediction=state_path).assign(Delta=state_delta))
    print()
# Tell python to run main method
if __name__ == "__main__": main()

--- Hidden states ---
Present    0.32
Absent     0.68
Name: states, dtype: float64

--- Transition matrix for hidden states ---
        Present Absent
Present    0.75   0.25
Absent     0.25   0.75

Present    1.0
Absent     1.0
dtype: float64

--- Sensor matrix ---
        BoringClass InterestingClass
Present         0.1              0.9
Absent          0.8              0.2

Present    1.0
Absent     1.0
dtype: float64

--- Hidden edges ---
{('Absent', 'Absent'): 0.75,
 ('Absent', 'Present'): 0.25,
 ('Present', 'Absent'): 0.25,
 ('Present', 'Present'): 0.75}

--- Sensor edges ---
{('Absent', 'BoringClass'): 0.8,
 ('Absent', 'InterestingClass'): 0.2,
 ('Present', 'BoringClass'): 0.1,
 ('Present', 'InterestingClass'): 0.9}

--- Predictions ---
        Observation Prediction     Delta
0  InterestingClass    Present  0.288000
1  InterestingClass    Present  0.194400
2  InterestingClass    Present  0.131220
3       BoringClass    Present  0.026244
4  InterestingClass    Present  0.006643
5 

### **Rainy/Sunny Predictor**

In [ ]:
!pip install pomegranate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.3 MB 4.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pomegranate: filename=pomegranate-0.14.8-cp37-cp37m-linux_x86_64.whl size=15066188 sha256=b4714256bb35b961e7ec3a82e42a1b606e869b55414a720cbab285b7244af6be
  Stored in directory: /root/.cache/pip/wheels/24/68/69/0eaab474ef1d65abedcd47de8a38ab21d221d329954d7edd24
Successfully built pomegranate


In [ ]:
from pomegranate import *
import random
import math

random.seed(0)

model = HiddenMarkovModel( name="Rainy-Sunny" )

rainy = State( DiscreteDistribution({ 'walk': 0.1, 'shop': 0.4, 'clean': 0.5 }), name='Rainy' )
sunny = State( DiscreteDistribution({ 'walk': 0.6, 'shop': 0.3, 'clean': 0.1 }), name='Sunny' )

model.add_transition( model.start, rainy, 0.6 )
model.add_transition( model.start, sunny, 0.4 )

model.add_transition( rainy, rainy, 0.65 )
model.add_transition( rainy, sunny, 0.25 )
model.add_transition( sunny, rainy, 0.35 )
model.add_transition( sunny, sunny, 0.55 )

model.add_transition( rainy, model.end, 0.1 )
model.add_transition( sunny, model.end, 0.1 )

model.bake( verbose=True )
sequence = [ 'walk', 'shop', 'clean', 'clean', 'clean', 'walk', 'clean' ]
print(math.e**model.forward( sequence )[ len(sequence), model.end_index ])
print(math.e**model.forward_backward( sequence )[1][ 2, model.states.index( rainy ) ])

1.854526990800002e-05
0.9120990704186626


In [ ]:
print(math.e**model.backward( sequence )[ 3, model.states.index( sunny ) ])
print(" ".join( state.name for i, state in model.maximum_a_posteriori( sequence )[1] ))

0.0004459435000000002
Sunny Rainy Rainy Rainy Rainy Sunny Rainy
